In [2]:
import pandas as pd
import numpy as np

import yaml

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel


In [4]:
with open("../../config.yaml", "r") as stream:
    try:
        configs = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [7]:
TT = 'train' #(T)rain or (T)est
PATH_TO_EXTRACTED_FEATURES = configs['ROOT_DIR'] + configs['PATH_TO_EXTRACTED_FEATURES']
PATH_TO_TRAIN_DS_PROCESSED = str(configs['ROOT_DIR'] + configs['PATH_TO_TRAIN_DS_PROCESSED'])
PATH_TO_MODELS             = str(configs['ROOT_DIR'] + configs['PATH_TO_MODELS'])

'B:\\utils\\Documentos\\MDC\\projeto_final\\meme-sentiment-analysis/datasets/processed/train.csv'

In [6]:
base = pd.read_csv(PATH_TO_TRAIN_DS_PROCESSED)
base.head()

,image_name,text,humour,sarcasm,offensive,motivational,target
0,image_1415.jpeg,if you want to view paradis simpli look around...,not_funny,not_sarcastic,not_offensive,motivational,positive
1,image_6460.png,if i had a brick for everi lie hillari told i ...,hilarious,general,very_offensive,motivational,positive
2,image_2303.png,that thing over there can i eat that,very_funny,general,not_offensive,motivational,very_positive
3,image_2417.png,my dad point to liam and said when did david b...,not_funny,general,very_offensive,motivational,neutral
4,image_11.jpg,probabl the first man to do year challeng chen...,funny,general,very_offensive,motivational,negative


## RoBERTa

In [8]:
model_name = "roberta-base"

In [148]:
tok = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(PATH_TO_MODELS + model_name + '/')

Some layers from the model checkpoint at B:\utils\Documentos\MDC\projeto_final\meme-sentiment-analysis/meme-sentiment-analysis/models/saved/models/roberta-base/ were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaModel were not initialized from the model checkpoint at B:\utils\Documentos\MDC\projeto_final\meme-sentiment-analysis/meme-sentiment-analysis/models/saved/models/roberta-base/ and are newly initialized: ['roberta/pooler/dense/kernel:0', 'roberta/pooler/dense/bias:0']
You should probably T

In [149]:
extracted_features = np.zeros(768)
extracted_features.shape = (1, 768)

for i in range(0, len(base.text.values.tolist()) - 100, 100):
  input_ids = tf.constant(tok(base.text.values.tolist()[ i : i + 100], padding='max_length', truncation=True)['input_ids'])
  outputs = model(input_ids)
  last_hidden_states = outputs[0]
  embeddings = last_hidden_states.numpy()[:, 0, :]
  extracted_features = np.concatenate((extracted_features, embeddings), axis=0)
  
input_ids = tf.constant(tok(base.text.values.tolist()[ i + 100 : len(base.text.values.tolist()) + 1], padding='max_length', truncation=True)['input_ids'])
outputs = model(input_ids)
last_hidden_states = outputs[0]
embeddings = last_hidden_states.numpy()[:, 0, :]
extracted_features = np.concatenate((extracted_features, embeddings), axis=0)

In [151]:
pd.DataFrame(extracted_features).to_csv(PATH_TO_EXTRACTED_FEATURES + model_name + "-" + TT + ".csv")